In [3]:
import requests
import pandas as pd
import numpy as np
import streamlit as st


FMP_API_KEY = st.secrets["FMP_API_KEY"] # replace with your Financial Modeling Prep API key
ALPHA_API_KEY = st.secrets["ALPHA_API_KEY"] # replace with your Alpha Vantage API key


In [7]:
def get_company_info(symbol: str) -> dict:
    """
    Returns a dictionary containing information about a company with the given stock symbol.
    
    Parameters:
        symbol (str): Stock symbol
        
    Returns:
        dict: Dictionary containing information about the company
    """
    # Set the API endpoint URL
    api_endpoint = f'https://financialmodelingprep.com/api/v3/profile/{symbol}/'
    params = {
        'apikey': FMP_API_KEY,  
    }

    try:
        # Make an HTTP GET request to the API with the specified parameters
        response = requests.get(api_endpoint, params=params)

        # Check for any errors in the HTTP response status code
        response.raise_for_status()

        # Convert the response data to a Python dictionary
        data = response.json()

        # Extract the first (and only) item from the list of company data
        data = data[0]

        # Extract the desired company information from the dictionary
        company_info = {
            'Name': data['companyName'],
            'Exchange': data['exchangeShortName'],
            'Currency': data['currency'],
            'Country': data['country'],            
            'Sector': data['sector'],
            'Market Cap':  data['mktCap'],
            'Price': data['price'],
            'Beta': data['beta'],
            'Price change': data['changes'],
            'Website': data['website'],
            'Image': data['image']
        }

        # Return the company information dictionary
        return company_info

    except requests.exceptions.RequestException as e:
        # Handle any errors that occur while making the API request
        print(f"Error occurred while fetching data from API: {e}")
        return None

    except ValueError as e:
        # Handle any errors that occur while parsing the response data
        print(f"Error occurred while parsing JSON response: {e}")
        return None


In [8]:
get_company_info('AAPL')

{'Name': 'Apple Inc.',
 'Exchange': 'NASDAQ',
 'Currency': 'USD',
 'Country': 'US',
 'Sector': 'Technology',
 'Market Cap': 3470720194000,
 'Price': 226.34,
 'Beta': 1.244,
 'Price change': 4.79,
 'Website': 'https://www.apple.com',
 'Image': 'https://financialmodelingprep.com/image-stock/AAPL.png'}

In [14]:
def get_stock_price(symbol: str) -> pd.DataFrame:
    """
    Returns a Pandas DataFrame containing the monthly adjusted closing prices of a given stock symbol
    for the last 5 years.
    
    Parameters:
        symbol (str): Stock symbol
        
    Returns:
        pd.DataFrame: Pandas DataFrame containing the monthly adjusted closing prices of the stock
    """
    api_endpoint = 'https://www.alphavantage.co/query'
    params = {
        'function': 'TIME_SERIES_MONTHLY_ADJUSTED',
        'symbol': symbol,  
        'apikey': ALPHA_API_KEY,  
    }

    try:
        # Make an HTTP GET request to the API
        response = requests.get(api_endpoint, params=params)
        response.raise_for_status()  # raise exception for any bad HTTP status code

        # Parse the response JSON data into a Pandas DataFrame
        data = response.json()['Monthly Adjusted Time Series']
        df = pd.DataFrame.from_dict(data, orient='index')
        df.index = pd.to_datetime(df.index)
        df = df[:12*5] # get data for the last 5 years
        df = df[['4. close']].astype(float)
        df = df.rename(columns={'4. close': 'Price'})

        return df

    except requests.exceptions.RequestException as e:
        print(f"Error occurred while fetching data from API: {e}")
        return None

    except ValueError as e:
        print(f"Error occurred while parsing JSON response: {e}")
        return None

In [15]:
get_stock_price("AAPL")

,Price
2024-07-05,226.34
2024-06-28,210.62
2024-05-31,192.25
2024-04-30,170.33
2024-03-28,171.48
2024-02-29,180.75
2024-01-31,184.40
2023-12-29,192.53
2023-11-30,189.95
2023-10-31,170.77


In [16]:
def get_income_statement(symbol: str) -> pd.DataFrame:
    """
    Retrieves the income statement data for a given stock symbol from the Financial Modeling Prep API.

    Args:
        symbol (str): The stock symbol to retrieve the income statement data for.

    Returns:
        pd.DataFrame: A Pandas DataFrame containing the income statement data.
    """
    # define the API endpoint and parameters
    api_endpoint = f'https://financialmodelingprep.com/api/v3/income-statement/{symbol}/'
    params = {
        'limit': 5,
        'apikey': FMP_API_KEY,  
    }
    try:
        # create an empty list to store the income statement data
        income_statement_data = []

        # make an HTTP GET request to the API
        response = requests.get(api_endpoint, params=params)
        response.raise_for_status()  # raise exception for any bad HTTP status code

        # parse the response JSON data into a list of dictionaries
        response_data = response.json()

        # extract the income statement data from the list of dictionaries
        for report in response_data:
            year = report['calendarYear'] 
            income_statement_data.append({
                'Year': year,
                'Revenue': (report['revenue']),
                '(-) Cost of Revenue': (report['costOfRevenue']),
                '= Gross Profit': (report['grossProfit']),
                '(-) Operating Expense': (report['operatingExpenses']),
                '= Operating Income': (report['operatingIncome']),
                '(+-) Other Income/Expenses': (report['totalOtherIncomeExpensesNet']),
                '= Income Before Tax': (report['incomeBeforeTax']),                
                '(+-) Tax Income/Expense': (report['incomeTaxExpense']),
                '= Net Income': (report['netIncome']),
            })

        # create a Pandas DataFrame from the list of dictionaries and return it
        income_statement = pd.DataFrame(income_statement_data).set_index('Year')

        return income_statement
        
    except requests.exceptions.RequestException as e:
        print(f"Error occurred while fetching data from API: {e}")
        return None

    except ValueError as e:
        print(f"Error occurred while parsing JSON response: {e}")
        return None


def get_balance_sheet(symbol: str) -> pd.DataFrame:
    """
    Retrieves the balance sheet data for a given stock symbol.

    Args:
        symbol (str): Stock symbol to retrieve balance sheet data for.

    Returns:
        pd.DataFrame: Pandas DataFrame containing the balance sheet data.
    """
    # Define the API endpoint and parameters
    api_endpoint = f'https://financialmodelingprep.com/api/v3/balance-sheet-statement/{symbol}'
    params = {
        'limit': 10,
        'apikey': FMP_API_KEY,  
    }

    try:
        # Create an empty list to store the balance sheet data
        balance_sheet_data = []

        # Make an HTTP GET request to the API
        response = requests.get(api_endpoint, params=params)
        response.raise_for_status()  # Raise exception for any bad HTTP status code

        # Parse the response JSON data into a list of dictionaries
        response_data = response.json()

        # Extract the balance sheet data from the list of dictionaries
        for report in response_data:
            year = report['calendarYear']
            balance_sheet_data.append({
                'Year': year,
                'Assets': report['totalAssets'],
                'Current Assets': report['totalCurrentAssets'],
                'Non-Current Assets': report['totalNonCurrentAssets'],
                'Current Liabilities': report['totalCurrentLiabilities'],
                'Non-Current Liabilities': report['totalNonCurrentLiabilities'],
                'Liabilities': report['totalLiabilities'],
                'Equity': report['totalEquity']
            })

        # Create a Pandas DataFrame from the list of dictionaries and return it
        balance_sheet_df = pd.DataFrame(balance_sheet_data).set_index('Year')

        return balance_sheet_df
    
    except requests.exceptions.RequestException as e:
        # If an error occurs, print the error message and return None
        print('Error getting balance sheet data:', e)
        return None

In [17]:
get_income_statement("AAPL")

,Revenue,(-) Cost of Revenue,= Gross Profit,(-) Operating Expense,= Operating Income,(+-) Other Income/Expenses,= Income Before Tax,(+-) Tax Income/Expense,= Net Income
Year,,,,,,,,,
2023,383285000000,214137000000,169148000000,54847000000,114301000000,-565000000,113736000000,16741000000,96995000000
2022,394328000000,223546000000,170782000000,51345000000,119437000000,-334000000,119103000000,19300000000,99803000000
2021,365817000000,212981000000,152836000000,43887000000,108949000000,258000000,109207000000,14527000000,94680000000
2020,274515000000,169559000000,104956000000,38668000000,66288000000,803000000,67091000000,9680000000,57411000000
2019,260174000000,161782000000,98392000000,34462000000,63930000000,1807000000,65737000000,10481000000,55256000000


In [21]:
def get_balance_sheet(symbol: str) -> pd.DataFrame:
    """
    Retrieves the balance sheet data for a given stock symbol.

    Args:
        symbol (str): Stock symbol to retrieve balance sheet data for.

    Returns:
        pd.DataFrame: Pandas DataFrame containing the balance sheet data.
    """
    # Define the API endpoint and parameters
    api_endpoint = f'https://financialmodelingprep.com/api/v3/balance-sheet-statement/{symbol}'
    params = {
        'limit': 5,
        'apikey': FMP_API_KEY,  
    }

    try:
        # Create an empty list to store the balance sheet data
        balance_sheet_data = []

        # Make an HTTP GET request to the API
        response = requests.get(api_endpoint, params=params)
        response.raise_for_status()  # Raise exception for any bad HTTP status code

        # Parse the response JSON data into a list of dictionaries
        response_data = response.json()

        # Extract the balance sheet data from the list of dictionaries
        for report in response_data:
            year = report['calendarYear']
            balance_sheet_data.append({
                'Year': year,
                'Assets': report['totalAssets'],
                'Current Assets': report['totalCurrentAssets'],
                'Non-Current Assets': report['totalNonCurrentAssets'],
                'Current Liabilities': report['totalCurrentLiabilities'],
                'Non-Current Liabilities': report['totalNonCurrentLiabilities'],
                'Liabilities': report['totalLiabilities'],
                'Equity': report['totalEquity']
            })

        # Create a Pandas DataFrame from the list of dictionaries and return it
        balance_sheet_df = pd.DataFrame(balance_sheet_data).set_index('Year')

        return balance_sheet_df
    
    except requests.exceptions.RequestException as e:
        # If an error occurs, print the error message and return None
        print('Error getting balance sheet data:', e)
        return None

In [22]:
get_balance_sheet("AAPL")

,Assets,Current Assets,Non-Current Assets,Current Liabilities,Non-Current Liabilities,Liabilities,Equity
Year,,,,,,,
2023,352583000000,143566000000,209017000000,145308000000,145129000000,290437000000,62146000000
2022,352755000000,135405000000,217350000000,153982000000,148101000000,302083000000,50672000000
2021,351002000000,134836000000,216166000000,125481000000,162431000000,287912000000,63090000000
2020,323888000000,143713000000,180175000000,105392000000,153157000000,258549000000,65339000000
2019,338516000000,162819000000,175697000000,105718000000,142310000000,248028000000,90488000000


In [20]:
def get_cash_flow(symbol: str) -> pd.DataFrame:
    """
    Retrieve cash flow data for a given stock symbol from the Financial Modeling Prep API.
    
    Args:
        symbol (str): The stock symbol for the company.
    
    Returns:
        pd.DataFrame: A Pandas DataFrame containing cash flow data for the company.
    """
    # Define the API endpoint and parameters
    api_endpoint = f'https://financialmodelingprep.com/api/v3/cash-flow-statement/{symbol}'
    params = {
        'limit': 5,
        'apikey': FMP_API_KEY,  
    }
    
    try:
        # Create an empty list to store the cash flow data
        cashflow_data = []

        # Make an HTTP GET request to the API
        response = requests.get(api_endpoint, params=params)
        response.raise_for_status()  # Raise an exception for any bad HTTP status code

        # Parse the response JSON data into a list of dictionaries
        response_data = response.json()

        # Extract the cash flow data from the list of dictionaries
        for report in response_data:
            year = report['date'].split('-')[0]
            cashflow_data.append({
                'Year': year,
                "Cash flows from operating activities": report['netCashProvidedByOperatingActivities'],
                'Cash flows from investing activities': report['netCashUsedForInvestingActivites'],
                'Cash flows from financing activities': report['netCashUsedProvidedByFinancingActivities'],
                'Free cash flow': report['freeCashFlow']
            })

        # Create a Pandas DataFrame from the list of dictionaries and set the 'Year' column as the index
        cashflow_df = pd.DataFrame(cashflow_data).set_index('Year')

        return cashflow_df
    
    except requests.exceptions.RequestException as e:
        # If an error occurs, print the error message and return None
        print('Error getting cash flow data:', e)
        return None

In [21]:
get_cash_flow("AAPL")

,Cash flows from operating activities,Cash flows from investing activities,Cash flows from financing activities,Free cash flow
Year,,,,
2023,110543000000,3705000000,-108488000000,99584000000
2022,122151000000,-22354000000,-110749000000,111443000000
2021,104038000000,-14545000000,-93353000000,92953000000
2020,80674000000,-4289000000,-86820000000,73365000000
2019,69391000000,45896000000,-90976000000,58896000000


In [9]:
def get_key_metrics(symbol: str) -> pd.DataFrame:
    """
    Returns a Pandas DataFrame containing the key financial metrics of a given company symbol for the last 10 years.

    Parameters:
        symbol (str): Company symbol.

    Returns:
        pd.DataFrame: Pandas DataFrame containing the key financial metrics.
    """
    # Define the API endpoint and parameters.
    api_endpoint = f'https://financialmodelingprep.com/api/v3/key-metrics/{symbol}'
    params = {
        'limit': 5,  # Get data for the last 5 years.
        'apikey': FMP_API_KEY, 
    }

    try:
        # Create an empty list to store the ratios data.
        metrics_data = []

        # Make an HTTP GET request to the API.
        response = requests.get(api_endpoint, params=params)
        response.raise_for_status()  # Raise exception for any bad HTTP status code.

        # Parse the response JSON data into a list of dictionaries.
        response_data = response.json()

        # Extract the ratios data from the list of dictionaries.
        for report in response_data:
            year = report['date'].split('-')[0]  # Extract the year from the date string.
            metrics_data.append({
                'Year': year,
                "Market Cap": report['marketCap'],
                'Working Capital': report['workingCapital'],
                'D/E ratio': report['debtToEquity'],
                'P/E Ratio': report['peRatio'],
                'ROE': report['roe'], 
                'Dividend Yield': report['dividendYield']
            })

        # Create a Pandas DataFrame from the list of dictionaries and return it.
        metrics_df = pd.DataFrame(metrics_data).set_index('Year')
        return metrics_df
    
    except requests.exceptions.RequestException as e:
        # If an error occurs, print the error message and return None.
        print(f"Error occurred while fetching data from API: {e}")
        return None

In [10]:
get_key_metrics("AAPL")

,Market Cap,Working Capital,D/E ratio,P/E Ratio,ROE,Dividend Yield
Year,,,,,,
2023,2695569789510,-1742000000,1.787533,27.790812,1.560760,0.005574
2022,2439367314090,-18577000000,2.369533,24.441824,1.969589,0.006084
2021,2453750882240,9355000000,1.976843,25.916254,1.500713,0.005896
2020,1948295921320,38321000000,1.720810,33.935934,0.878664,0.007227
2019,1010566792560,57101000000,1.194048,18.288816,0.610645,0.013971


In [4]:
def get_esg_data(symbol: str) -> pd.DataFrame:
    """
    Returns a Pandas DataFrame containing the ESG (Environmental, Social, Governance) data for a given company symbol.

    Parameters:
        symbol (str): Company symbol.

    Returns:
        pd.DataFrame: Pandas DataFrame containing the ESG data.
    """
    # Define the API endpoint and parameters.
    api_endpoint = f'https://financialmodelingprep.com/api/v4/esg-environmental-social-governance-data'
    params = {
        'symbol': symbol,
        'apikey': FMP_API_KEY, 
    }

    try:
        # Make an HTTP GET request to the API.
        response = requests.get(api_endpoint, params=params)
        response.raise_for_status()  # Raise exception for any bad HTTP status code.

        # Parse the response JSON data into a list of dictionaries.
        response_data = response.json()

        # Check if response data is empty or not
        if not response_data:
            print(f"No ESG data found for symbol: {symbol}")
            return pd.DataFrame()

        # Extract the relevant ESG data into a list of dictionaries.
        esg_data = []
        for report in response_data:
            year = report['date'].split('-')[0]  # Extract the year from the date string.
            esg_data.append({
                'Date': year,
                'Environmental Score': report['environmentalScore'],
                'Social Score': report['socialScore'],
                'Governance Score': report['governanceScore'],
                'Total ESG Score': report['ESGScore']
            })

        # Create a Pandas DataFrame from the list of dictionaries and return it.
        esg_df = pd.DataFrame(esg_data).set_index('Date')
        return esg_df
    
    except requests.exceptions.RequestException as e:
        # If an error occurs, print the error message and return None.
        print(f"Error occurred while fetching data from API: {e}")
        return None

In [5]:
get_esg_data('AAPL')

Error occurred while fetching data from API: 403 Client Error: Forbidden for url: https://financialmodelingprep.com/api/v4/esg-environmental-social-governance-data?symbol=AAPL&apikey=WmOu4FT93k418zz49C0gtGzAE55NP6Oi


In [4]:
# Import necessary libraries
import streamlit as st
from io import BytesIO
from millify import millify
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from io import BytesIO
import sys
from utils import (
    config_menu_footer, generate_card, empty_lines, get_delta, color_highlighter
)
from data import (
    get_income_statement, get_balance_sheet, get_stock_price, get_company_info,
    get_financial_ratios, get_key_metrics, get_cash_flow
)

from streamlit_option_menu import option_menu

# Define caching functions for each API call
@st.cache_data(ttl=60*60*24*30) # cache output for 30 days
def company_info(symbol):
    return get_company_info(symbol)

@st.cache_data(ttl=60*60*24*30) # cache output for 30 days
def income_statement(symbol):
    return get_income_statement(symbol)

@st.cache_data(ttl=60*60*24*30) # cache output for 30 days
def balance_sheet(symbol):
    return get_balance_sheet(symbol)

@st.cache_data(ttl=60*60*24*30) # cache output for 30 days
def stock_price(symbol):
    return get_stock_price(symbol)

@st.cache_data(ttl=60*60*24*30) # cache output for 30 days
def financial_ratios(symbol):
    return get_financial_ratios(symbol)

@st.cache_data(ttl=60*60*24*30) # cache output for 30 days
def key_metrics(symbol):
    return get_key_metrics(symbol)

@st.cache_data(ttl=60*60*24*30) # cache output for 30 days
def cash_flow(symbol):
    return get_cash_flow(symbol)


# Configure the app page
st.set_page_config(
    page_title='Financial Report',
    page_icon='📈',
    layout="centered",
)

# Define caching function for delta
@st.cache_data(ttl=60*60*24*30) # cache output for 30 days
def delta(df,key):
    return get_delta(df,key)

# Configure the menu and footer with the user's information
config_menu_footer()


# # customize background
# page_bg_img = f"""
# <style>
# [data-testid="stAppViewContainer"] > .main {{
# background-image: url("https://wallpapercave.com/wp/wp3075569.jpg");
# background-size: 100%;
# background-position: top left;
# background-attachment: local;
# }}


# [data-testid="stHeader"] {{
# background: rgba(0,0,0,0);
# }}

# [data-testid="stToolbar"] {{
# right: 2rem;
# }}
# </style>
# """
# #display background
# st.markdown(page_bg_img, unsafe_allow_html=True)

# Display the app title
st.title("Financial Report 📈")

# add slidebar
with st.sidebar:
    selected = option_menu(
        menu_title="Menu",
        options=["Financial Report", "Watch list", "Financial Tutorial"],
        icons=["search", "list", "book"],
        default_index=0,
    )


# Initialize the state of the button as False when the app is first loaded
if 'btn_clicked' not in st.session_state:
    st.session_state['btn_clicked'] = False

# Define a callback function for when the "Go" button is clicked
def callback():
    # change state value
    st.session_state['btn_clicked'] = True

# Create a text input field for the user to enter a stock ticker
symbol_input = st.text_input("Enter a stock ticker").upper()

# Check if the "Go" button has been clicked
if st.button('Go',on_click=callback) or st.session_state['btn_clicked']:
    
    # Check if the user has entered a valid ticker symbol
    if not symbol_input:
        st.warning('Please input a ticker.')
        st.stop()

    try:
        # Call the API functions to get the necessary data for the dashboard
        company_data = get_company_info(symbol_input)
        metrics_data = key_metrics(symbol_input)
        income_data = income_statement(symbol_input)
        performance_data = stock_price(symbol_input)
        ratios_data = financial_ratios(symbol_input)
        balance_sheet_data = balance_sheet(symbol_input)
        cashflow_data = cash_flow(symbol_input)

    except Exception:
        st.error('Not possible to retrieve data for that ticker. Please check if its valid and try again.')
        sys.exit()

    # Display dashboard
    empty_lines(2)
    try:
        # Display company info
        col1, col2 = st.columns((8.5,1.5))
        with col1:
            generate_card(company_data['Name'])
        with col2:
            # display image and make it clickable
            image_html = f"<a href='{company_data['Website']}' target='_blank'><img src='{company_data['Image']}' alt='{company_data['Name']}' height='95' width='95'></a>"
            st.markdown(image_html, unsafe_allow_html=True)

        col3, col4, col5, col6, col7 = st.columns((0.2,1.4,1.4,2,2.6))

        with col4:
            empty_lines(1)
            st.metric(label="Price", value=company_data['Price'], delta=company_data['Price change'])
            empty_lines(2)

        with col5:
            empty_lines(1)
            generate_card(company_data['Currency'])
            empty_lines(2)

        with col6:
            empty_lines(1)
            generate_card(company_data['Exchange'])
            empty_lines(2)

        with col7:
            empty_lines(1)
            generate_card(company_data['Sector'])            
            empty_lines(2)

        # Define columns for key metrics and IS
        col8, col9, col10 = st.columns((2,2,3))

        # Display key metrics  
        with col8:
            empty_lines(3)
            st.metric(label="Market Cap", value=millify(metrics_data['Market Cap'][0], precision=2), delta=delta(metrics_data,'Market Cap'))
            st.write("")
            st.metric(label="D/E Ratio", value = round(metrics_data['D/E ratio'][0],2), delta=delta(metrics_data,'D/E ratio'))
            st.write("")
            st.metric(label="ROE", value = str(round(metrics_data['ROE'][0] * 100, 2)) + '%', delta=delta(metrics_data,'ROE'))

        with col9:
            empty_lines(3)
            st.metric(label="Working Capital", value = millify(metrics_data['Working Capital'][0], precision = 2), delta=delta(metrics_data,'Working Capital'))
            st.write("")
            st.metric(label="P/E Ratio", value = round(metrics_data['P/E Ratio'][0],2), delta=delta(metrics_data,'P/E Ratio'))
            st.write("")
            # Check if the company pays dividends
            if metrics_data['Dividend Yield'][0] == 0:
                st.metric(label="Dividends (yield)", value = '0')
            else:
                st.metric(label="Dividends (yield)", value = str(round(metrics_data['Dividend Yield'][0]* 100, 2)) + '%', delta=delta(metrics_data,'Dividend Yield'))
        with col10:      
            # Transpose the income data so that the years are the columns
            income_statement_data = income_data.T

            # Display a markdown header for the income statement
            st.markdown('**Income Statement**')
                        
            # Allow the user to select a year to display
            year = st.selectbox('All numbers in thousands', income_statement_data.columns, label_visibility='collapsed')

            # Slice the income data to only show the selected year and format numbers with millify function
            income_statement_data = income_statement_data.loc[:, [year]]
            income_statement_data = income_statement_data.applymap(lambda x: millify(x, precision=2))
                        
            # Apply the color_highlighter function to highlight negative numbers
            income_statement_data = income_statement_data.style.applymap(color_highlighter)

            # Style the table headers with black color
            headers = {
                'selector': 'th:not(.index_name)',
                'props': [('color', 'black')]
            }

            income_statement_data.set_table_styles([headers])

            # Display the income statement table in Streamlit
            st.table(income_statement_data)


        # Configure the plots bar
        config = {
            'displaylogo': False, 
            'modeBarButtonsToRemove': ['zoom2d', 'pan2d', 'select2d', 'lasso2d', 'hoverClosestCartesian', 'hoverCompareCartesian', 'autoScale2d', 'toggleSpikelines', 'resetScale2d', 'zoomIn2d', 'zoomOut2d', 'hoverClosest3d', 'hoverClosestGeo', 'hoverClosestGl2d', 'hoverClosestPie', 'toggleHover', 'resetViews', 'toggleSpikeLines', 'resetViewMapbox', 'resetGeo', 'hoverClosestGeo', 'sendDataToCloud', 'hoverClosestGl']
        }

        # Display market performance
        # Determine the color of the line based on the first and last prices
        line_color = 'rgb(60, 179, 113)' if performance_data.iloc[0]['Price'] > performance_data.iloc[-1]['Price'] else 'rgb(255, 87, 48)'

        # Create the line chart 
        fig = go.Figure(
            go.Scatter(
                x=performance_data.index,
                y=performance_data['Price'],
                mode='lines',
                name='Price',
                line=dict(color=line_color)
            )
        )

        # Customize the chart layout
        fig.update_layout(
            title={
                'text': 'Market Performance',
            },
            dragmode='pan',
            xaxis=dict(
            fixedrange=True
            ),
        yaxis=dict(
            fixedrange=True
            )
        )

        # Render the line chart 
        st.plotly_chart(fig, config=config, use_container_width=True)


        # Display net income
        # Create the line chart 
        fig = go.Figure()
        fig.add_trace(
            go.Scatter(
                x=income_data.index, 
                y=income_data["= Net Income"], 
                mode="lines+markers", 
                line=dict(
                    color="purple"), 
                    marker=dict(
                        size=5
                    )
                )
            )

        # Customize the chart layout
        fig.update_layout(
            title="Net Income",
            dragmode='pan',
            xaxis=dict(
                tickmode='array', 
                tickvals=income_data.index,
                fixedrange=True
            ),
            yaxis=dict(
                fixedrange=True
            ),
        )

        # Display the graph
        st.plotly_chart(fig, config=config, use_container_width=True)


        # Display profitability margins
        # Create an horizontal bar chart of profitability margins
        fig = go.Figure()
        fig.add_trace(go.Bar(
            y=ratios_data.index,
            x=ratios_data['Gross Profit Margin'],
            name='Gross Profit Margin',
            marker=dict(color='rgba(60, 179, 113, 0.85)'),
            orientation='h',
        ))
        fig.add_trace(go.Bar(
            y=ratios_data.index,
            x=ratios_data['Operating Profit Margin'],
            name='EBIT Margin',
            marker=dict(color='rgba(30, 144, 255, 0.85)'),
            orientation='h',
        ))

        fig.add_trace(go.Bar(
            y=ratios_data.index,
            x=ratios_data['Net Profit Margin'],
            name='Net Profit Margin',
            marker=dict(color='rgba(173, 216, 230, 0.85)'),
            orientation='h',
        ))

        # Update layout
        fig.update_layout(
            title='Profitability Margins',
            bargap=0.1,
            dragmode='pan',
            xaxis=dict(
                fixedrange=True,
                tickformat='.0%'
            ),
            yaxis=dict(
            fixedrange=True
             )
        )

        # Display the plot 
        st.plotly_chart(fig, config=config, use_container_width=True)


        #Display balance sheet
        # Create a vertical bar chart of Assets and Liabilities
        fig = go.Figure()
        fig.add_trace(go.Bar(
            x=balance_sheet_data.index,
            y=balance_sheet_data['Assets'],
            name='Assets',
            marker=dict(color='rgba(60, 179, 113, 0.85)'),
            width=0.3,
        ))
        fig.add_trace(go.Bar(
            x=balance_sheet_data.index,
            y=balance_sheet_data['Liabilities'],
            name='Liabilities',
            marker=dict(color='rgba(255, 99, 71, 0.85)'),
            width=0.3,
        ))

        # Add a line for assets
        fig.add_trace(go.Scatter(
            x=balance_sheet_data.index,
            y=balance_sheet_data['Equity'],
            mode='lines+markers',
            name='Equity',
            line=dict(color='rgba(173, 216, 230, 1)', width=2),
            marker=dict(symbol='circle', size=8, color='rgba(173, 216, 230, 1)', line=dict(width=1, color='rgba(173, 216, 230, 1)'))
        ))

        # Update layout
        fig.update_layout(
            title='Balance Sheet',
            bargap=0.4,
            dragmode='pan',
            xaxis=dict(
                fixedrange=True
            ),
                yaxis=dict(
                fixedrange=True,
             )
        )

        # Display the plot 
        st.plotly_chart(fig, config=config, use_container_width=True)


        # Display ROE and ROA
        # Create the line chart 
        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=ratios_data.index,
            y=ratios_data['Return on Equity'],
            name='ROE',
            line=dict(color='rgba(60, 179, 113, 0.85)'),
        ))
        fig.add_trace(go.Scatter(
            x=ratios_data.index,
            y=ratios_data['Return on Assets'],
            name='ROA',
            line=dict(color='rgba(30, 144, 255, 0.85)'),
        ))

        # Update layout
        fig.update_layout(
            title='ROE and ROA',
            dragmode='pan',
            xaxis=dict(
                fixedrange=True
            ),
            yaxis=dict(
                fixedrange=True,
                tickformat='.0%'
            )
        )

        # Display the plot in Streamlit
        st.plotly_chart(fig, config=config, use_container_width=True)


        # Display cash flows
        # Create a vertical bar chart of Cash flows
        fig = go.Figure()
        fig.add_trace(go.Bar(
            x=cashflow_data.index,
            y=cashflow_data['Cash flows from operating activities'],
            name='Cash flows from operating activities',
            marker=dict(color='rgba(60, 179, 113, 0.85)'),
            width=0.3,
        ))
        fig.add_trace(go.Bar(
            x=cashflow_data.index,
            y=cashflow_data['Cash flows from investing activities'],
            name='Cash flows from investing activities',
            marker=dict(color='rgba(30, 144, 255, 0.85)'),
            width=0.3,
        ))

        fig.add_trace(go.Bar(
            x=cashflow_data.index,
            y=cashflow_data['Cash flows from financing activities'],
            name='Cash flows from financing activities',
            marker=dict(color='rgba(173, 216, 230, 0.85)'),
            width=0.3,
        ))

        # Add a line for Free cash flow
        fig.add_trace(go.Scatter(
            x=cashflow_data.index,
            y=cashflow_data['Free cash flow'],
            mode='lines+markers',
            name='Free cash flow',
            line=dict(color='rgba(255, 140, 0, 1)', width=2),
            marker=dict(symbol='circle', size=5, color='rgba(255, 140, 0, 1)', line=dict(width=0.8, color='rgba(255, 140, 0, 1)'))
        ))

        # Update layout
        fig.update_layout(
            title='Cash flows',
            bargap=0.1,
                xaxis=dict(
                fixedrange=True,
            ),
            yaxis=dict(
                fixedrange=True,
            )
        )

        # Display the plot 
        st.plotly_chart(fig, config=config, use_container_width=True)

        #Display financial ratios table 
        empty_lines(1)
        st.markdown('**Financial Ratios**')
        # Rename keys and format values as needed
        ratios_table = ratios_data.rename(columns={
            'Days of Sales Outstanding': 'Days of Sales Outstanding (days)',
            'Days of Inventory Outstanding': 'Days of Inventory Outstanding (days)',
            'Operating Cycle': 'Operating Cycle (days)',
            'Days of Payables Outstanding': 'Days of Payables Outstanding (days)',
            'Cash Conversion Cycle': 'Cash Conversion Cycle (days)',
            'Gross Profit Margin': 'Gross Profit Margin (%)', 
            'Operating Profit Margin': 'Operating Profit Margin (%)',
            'Pretax Profit Margin': 'Pretax Profit Margin (%)',
            'Net Profit Margin': 'Net Profit Margin (%)',
            'Effective Tax Rate': 'Effective Tax Rate (%)',
            'Return on Assets': 'Return on Assets (%)',
            'Return on Equity': 'Return on Equity (%)',
            'Return on Capital Employed': 'Return on Capital Employed (%)',
            'EBIT per Revenue': 'EBIT per Revenue (%)',
            'Debt Ratio': 'Debt Ratio (%)',
            'Long-term Debt to Capitalization': 'Long-term Debt to Capitalization (%)',
            'Total Debt to Capitalization': 'Total Debt to Capitalization (%)',
            'Payout Ratio': 'Payout Ratio (%)',
            'Operating Cash Flow Sales Ratio': 'Operating Cash Flow Sales Ratio (%)',
            'Dividend Yield': 'Dividend Yield (%)',
        })

        # Multiply values in columns with "%" symbol by 100
        for col in ratios_table.columns:
            if "%" in col:
                ratios_table[col] = ratios_table[col] * 100

        ratios_table = round(ratios_table.T,2)
        

        ratios_table = ratios_table.sort_index(axis=1, ascending=True)

        # Display ratios table
        st.dataframe(ratios_table, width=800, height=400)

    except Exception as e:
        st.error('Not possible to develop dashboard. Please try again.')
        sys.exit()

    #Add download button
    empty_lines(3)
    try:
        # Create dataframes for each financial statement
        company_data = pd.DataFrame.from_dict(company_data, orient='index')
        company_data = (
            company_data.reset_index()
            .rename(columns={'index':'Key', 0:'Value'})
            .set_index('Key')
        )
        metrics_data = metrics_data.round(2).T
        income_data = income_data.round(2)
        ratios_data = ratios_data.round(2).T
        balance_sheet_data = balance_sheet_data.round(2).T
        cashflow_data = cashflow_data.T

        # Clean up income statement column names and transpose dataframe
        income_data.columns = income_data.columns.str.replace(r'[\/\(\)\-\+=]\s?', '', regex=True)
        income_data = income_data.T

        # Combine all dataframes into a dictionary
        dfs = {
            'Stock': company_data,
            'Market Performance': performance_data,    
            'Income Statement': income_data,
            'Balance Sheet': balance_sheet_data,
            'Cash flow': cashflow_data,
            'Key Metrics': metrics_data,
            'Financial Ratios': ratios_table
        }

        # Write the dataframes to an Excel file, with special formatting for the Market Performance sheet
        output = BytesIO()
        writer = pd.ExcelWriter(output, engine='xlsxwriter')
        for sheet_name, df in dfs.items():
            if sheet_name == 'Market Performance':
                # Rename index column and format date column
                df.index.name = 'Date'
                df = df.reset_index()
                df['Date'] = pd.to_datetime(df['Date']).dt.strftime('%Y-%m-%d')
                # Write dataframe to Excel sheet without index column
                df.to_excel(writer, sheet_name=sheet_name, index=False)
            else:
                # Write dataframe to Excel sheet with index column
                df.to_excel(writer, sheet_name=sheet_name, index=True)
            # Autofit columns in Excel sheet
            writer.sheets[sheet_name].autofit()

        # Close the Excel writer object
        writer.close()

        # Create a download button for the Excel file
        data = output.getvalue()
        st.download_button(
            label='Download ' + symbol_input + ' Financial Data (.xlsx)',
            data=data,
            file_name=symbol_input + '_financial_data.xlsx',
            mime='application/octet-stream'
        )
    except Exception:
        st.info('Data not available for download')


2024-07-08 16:42:07.227 No runtime found, using MemoryCacheStorageManager
2024-07-08 16:42:07.230 No runtime found, using MemoryCacheStorageManager
2024-07-08 16:42:07.231 No runtime found, using MemoryCacheStorageManager
2024-07-08 16:42:07.233 No runtime found, using MemoryCacheStorageManager
2024-07-08 16:42:07.237 No runtime found, using MemoryCacheStorageManager
2024-07-08 16:42:07.238 No runtime found, using MemoryCacheStorageManager
2024-07-08 16:42:07.238 No runtime found, using MemoryCacheStorageManager
2024-07-08 16:42:07.240 No runtime found, using MemoryCacheStorageManager


NameError: name 'data' is not defined